# Homework 2 - Hypothesis Testing

Homework setup such as imports are mentioned below.

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

## Part 1
### Null Hypothesis: Logo B is not better than Logo A.

In [ ]:
significance = 0.001

sample_success_a = 500
sample_success_b = 550
sample_size = 1000

successes = np.array([sample_success_a, sample_success_b])

stat, p_value = proportions_ztest(count=successes, nobs=sample_size,  alternative='two-sided')
# report
print('z_stat: %0.3f, p_value: %0.3f' % (stat, p_value))
if p_value > significance:
    print ("Null hyppthesis is proven to be incorrect.")
else:
    print ("Null hyppthesis is proven to be correct.")

In [ ]:
significance = 0.005

sample_success_a = 500
sample_success_b = 550
sample_size = 1000

successes = np.array([sample_success_a, sample_success_b])

stat, p_value = proportions_ztest(count=successes, nobs=sample_size,  alternative='two-sided')
# report
print('z_stat: %0.3f, p_value: %0.3f' % (stat, p_value))
if p_value > significance:
    print ("Null hypothesis is proven to be incorrect.")
else:
    print ("Null hypothesis is proven to be correct.")

This signifies that the test is accurate, and logo B is indeed better than logo A.

## Part 2

### Task 1: Import the Data
Use the pandas.read_csv() function to import the dataset. The data is contained in two files: realEstate1.csv and realEstate2.csv. After you import these files separately, concatenate them into one big dataframe. This pandas dataframe will be used for data exploration and linear regression.

In [ ]:
# Import Data
re1 = pd.read_csv("realEstate1.csv")
re2 = pd.read_csv("realEstate2.csv")

# Concatenate Dataframe
data_list = [re1, re2]
df = pd.concat(data_list, ignore_index=True)
display(df)

### Task 2: Clean the Data
There are 206 different variables associated with each of the 348 houses in this dataset. Skim them and try to get a rough understanding of what information this dataset contains. If you've never seen a real estate listing before, you might take a look at one on this website (Links to an external site.) to get a better sense of the meanings of the column headers in the dataset.

Only keep houses with List Price between 200,000 and 1,000,000 dollars. This is an arbitrary choice and we realize that some of you are high rollers, but for our purposes we'll consider the others as outliers.

Remove columns that you don't think contribute to the value of the house. This is a personal decision - what attributes of a house are important to you? You should at least keep the following variables since the questions below will use them: ['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 'PkgSpacs', 'PropType', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']

Check the datatypes and convert any numbers that were read as strings to numerical values. (Hint: You can use str.replace() (Links to an external site.) to work with strings.) If there are any categorical values you're interested in, then you should convert them to numerical values as we saw in the notes (the get_dummies function may help). In particular,

Convert 'TotSqf' to an integer.
Add a new column titled Prop_Type_SingleFamily that is set to 0 if the i-th listing is a condo or townhouse, or 1 if the i-th listing is a single family house.
Remove the listings with erroneous 'Longitude' (one has Longitude = 0) and 'Taxes' values (two have unreasonably large values).

In [ ]:
df = df[(df.LstPrice >= 200000) & (df.LstPrice <= 1000000)] # Filter Data
df.reset_index(inplace=True, drop=True) # Fix index

# Filter columns
df = df[['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 'PkgSpacs', 'PropType', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']]

# Fix dtypes
df.TotSqf = pd.Series(df.TotSqf).str.replace(',', '', regex=True)
df.TotSqf = pd.to_numeric(df.TotSqf)

Prop_Type_SingleFamily_data = []

for i in df['PropType']:
    if i == 'Single Family':
        Prop_Type_SingleFamily_data.append(1)
    else:
        Prop_Type_SingleFamily_data.append(0)

df['Prop_Type_SingleFamily'] = Prop_Type_SingleFamily_data

df = df[(df.Longitude != 0) & (df.Taxes <= 10000) & (df.Taxes >= 1000)] # Filter Data
df.reset_index(inplace=True, drop=True) # Fix index
df

### Task 3: Exploratory Data Analysis
Explore the dataset. Write a short description of the dataset describing the number of items, the number of variables and check to see if the values are reasonable.

In [ ]:
df.describe()

The dataset has exactly 254 entries and has 14 variables. I found that there was an entry with tax = 1, so I filtered the data above in Task 2, to have the lowest tax value as 1000. Apart from that the data seems to be pretty reasonable.

Make a bar chart showing the breakdown of the different types of houses (single family, townhouse, condo).

In [ ]:
# df.groupby('PropType').nunique().plot.bar()
df['PropType'].hist()

Compute the correlation matrix and use a heat map to visualize the correlation coefficients.

In [ ]:
corrM = df.corr()
plt.figure(figsize=(14,14))
sns.heatmap(corrM, annot=True, vmin=-1, vmax=1)

Make a scatter plot matrix to visualize the correlations. For the plot, only use a subset of the columns: ['Acres', 'LstPrice', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt'].

In [ ]:

pd.plotting.scatter_matrix(df[['Acres', 'LstPrice', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']], figsize=[14,14])

plt.show()

Sold prices, list prices and taxes are strongly correlated. The size of the property, i.e. 'TotSqf' is averagely correlated to sold prices, list prices and taxes. Total beds and total baths is roughly correlated with the prices, taxes and square footage of the property. This means that as the listing price goes up, all the other columns mentioned above also go up more or less.

### Task 4: Geospatial Plot
Two of the variables are the latitude and longitude of each listing. Salt Lake City is on this nice east-west, north south grid, so even a simple plot of latitude vs longitude makes sense. Create a scatterplot of these two variables. Use color to indicate the price of the house. How does the price depend on the house location?

Bonus: If you can, overlay the scatterplot on a map of the city. (This is challenging, and we didn't teach you how to do it, so you should do the other parts of the assignment first.)

What can you say about the relation between the location and the house price?

In [ ]:
geoplot = df.plot(x='Longitude', y='Latitude', c='LstPrice', colormap='viridis', figsize=[10,10], kind='scatter')
# For some reason, when I add color to this graph, it makes the x-axis label invisible.

It seems like the real estate prices increase as we go North mostly, but also East. The prices go down as we get farther from North and East side of Salt Lake City.

### Task 5: Simple Linear Regression
Make sure to `pip3 install statsmodels`.

Use the 'ols' function from the statsmodels (Links to an external site.) package to regress the Sold price on some of the other variables. Your model should be of the form Sold Price = beta_0 + beta_1 * x, where x is one of the other variables.

You'll find that the best predictor of sold price is the list price. Report the R-squared value for this model (SoldPrice ~ LstPrice) and give an interpretation for its meaning. Also give an interpretation of beta_1 for this model. Make a plot of list price vs. sold price and overlay the prediction coming from your regression model.

In [ ]:
result = smf.ols(formula='SoldPrice ~ TotBed', data=df).fit()
print(result.summary())

In [ ]:
result = smf.ols(formula='SoldPrice ~ TotSqf', data=df).fit()
print(result.summary())

In [ ]:
result = smf.ols(formula='SoldPrice ~ Taxes', data=df).fit()
print(result.summary())

In [ ]:
result = smf.ols(formula='SoldPrice ~ LstPrice', data=df).fit()
print(result.summary())

Based on `List Price ~ Sold Price` model, the r-squared value, i.e. `0.99` signifies that the model fits the data well. It means that the List Price is strongly correlated to Sold Price. Based on Beta_1 of this model, i.e. `0.94`, it signifies that the seller is selling the property a little less than what it was listed for.

In [ ]:
sns.regplot(x='LstPrice', y='SoldPrice', data=df)

### Task 6: Multiple Regression
Develop a multilinear regression model for house prices in this neighborhood. We could use this to come up with a list price for houses coming on the market, so do not include the list price in your model and, for now, ignore the categorical variable Prop_Type. Your model should be of the form:

Sold Price = beta_0 + beta_1 * x_1 + beta_2 * x_2 + ... + beta_n * x_n

where x_i are predictive variables. Which variables are the best predictors for the Sold Price?

In [ ]:
result = smf.ols(formula='SoldPrice ~ TotSqf + GaragCap + Longitude + Acres', data=df).fit()
print(result.summary())

Iteratively, having all the variables excluding the ones mentioned in the assignment, I found out that Total Sq. Ft., Garage Capacity, Longitude and Acres were the best variables for an accurate model. I determined this by checking the p-values of variables. Based on my research, a variable p-value should be under 0.005.

Q: Often the price per square foot for a house is advertised. Is this what the coefficient for TotSqf is measuring? Provide an interpretation for the coefficient for TotSqf.
A: Yes, that is precisely what the TotSqf coefficient is measuring. For adding each sq. ft. the price will increase by about 90.21 dollars.

Q: Estimate the value that each Garage space adds to a house.
A: Based on my model, each garage space will add about 42,030 dollars to a house. I am 79.9% sure about my answer.

Q: Does latitude or longitude have an impact on house price? Explain.
A: To a certain extent, longitude does affect the price. This data set doesn't have enough information that would be considered valid for this model, but based on the my intuition yes it will affect it as well, but not as much as longitude. I personally think that based on the coefficient of longitude, it means the for every increment degree of Longitude, the price increases by 2.3 million, which is not true in real life scenario as once we cross a point moving North, the prices will start to fall again. But if we hypothetically have another variable that can determine the boundaries of Salt Lake and compare it with Longitude, then it will be a good variable to have. But in this case, with the current variables, I think having Longitude could be a bad option if trying to predict a price outside the data set range. But if used for predicting values within the range of data, longitude is a good variable as it implies that as you get closer to downtown, the prices will increase.

Q: If we wanted to start a 'house flipping' company, we'd have to be able to do a better job of predicting the sold price than the list price does. How does your model compare?
A: Based on my model, the only way we can make a profit is by increasing garage space. The other factors cannot be changes being a house flipper in my understanding as you cannot control the other factors such as acres, sq. ft., etc. A house flipper just fixes up the existing property. The probability of this being accurate is just 79.9%.

### Task 7: Incorporating a Categorical Variable
From the first model, it would appear that Property type is significant in predicting the sold price. On the other hand, the second model indicates that when you take into account total square footage, property type is no longer predictive. Explain this. (Hint: there is a confounder lurking here.) Make a scatterplot of TotSqf vs. SoldPrice where the house types are colored differently to illustrate your explanation

In [ ]:
color_dict = {'Single Family': 'red', 'Condo': 'blue', 'Townhouse': 'green'}
# fig, ax = plt.subplots()
PropType = df.PropType.astype('category').cat.codes
plt.figure(figsize=(10,10))
scatter = plt.scatter(y=df.TotSqf, x=df.SoldPrice, c=PropType, s=10)
plt.xlabel('Sold Price')
plt.ylabel('Total Sq. Ft.')
prop_names = ['Condo', 'Single Family', 'Townhouse']
plt.legend(handles=scatter.legend_elements()[0],
           labels=prop_names,
           title="Property Type")
# df.plot(x='SoldPrice', y='TotSqf', c=[color_dict[i] for i in df.PropType], colormap='viridis', figsize=[10,10], kind='scatter')
#plt.legend(['Single Family', 'Condo', 'Townhouse'])
# For some reason, when I add color to this graph, it makes the x-axis label invisible.

Property type is no longer predictable because adding Total sq. ft. variables causes a spurious correlation. Sold price goes up regardless of the property type as the sq. ft. increases. There are not enough condos or townhouses in the range of comparison, therefore it can't be confidently assumed that the property will affect the regression.